In [1]:
# 0) 自動重載
%load_ext autoreload
%autoreload 2

import sys, pathlib
sys.path.append(str(pathlib.Path.cwd()))  # 確保可 import 專案模組

In [2]:
# 1) 匯入與基本參數
import numpy as np
import cudf
from collections import Counter
from solve import (
    build_waypoint_graph,graph_to_csr_arrays,
    choose_deliveries_near_first,
    setup_data_model, solve_routing,
    assert_route_indexed, compute_waypoint_arrival_times,
    print_all, apply_vehicle_breaks
)

from animation import build_animation_figure

from conflict import (
    make_waypoint_schedule,
    detect_conflicts,
    resolve_conflicts, 
    schedule_from_arrival_df,
)

setup variable -----------------------------

In [3]:
# ====== CONFIG（只改這裡就能切換實驗）======
CONFIG = {
    # 工廠時間窗
    "factory_open": 0,
    "factory_close": 400,

    # 取貨點與站點（原始 waypoint id）
    "pickup_node": 1,
    "stations": [10, 12, 13, 15, 16, 18, 19, 21, 22, 24, 25, 27, 28, 30, 31, 33],

    # 件數與服務時間
    "num_items": 16,
    "pickup_service_time": 0,
    "delivery_service_time": 0,

    # 車隊
    "robots": {
        "ids": [0, 1, 2, 3, 4, 5, 6, 7, 8],
        "capacity": [1, 1, 1 ,1, 1,1, 1, 1, 1],  # 每台車的載重能力
        # "earliest_start": [0, 1, 3, 6, 4],
        "earliest_start": [0, 1, 2, 3, 4, 5, 6, 7, 8],  # 每台車的最早開始時間
        "latest_end": [50, 50, 50, 50, 50, 50, 200, 150, 100],  # 每台車的最晚結束時間
    },

    # Solver 時間上限（秒）
    "solver_time_limit": 5,

    # 是否用「就近平均分配」規則自動產生 deliveries
    "use_nearby_split": True,

    # 是否要印出 choose_deliveries 的統計（每站件數與 ETA）
    "verbose_split": True,

    # 是否計算 waypoint 到達時間（需要 CSR 權重）
    "compute_wp_times": True,

    "charging_spots": [9],
    
    "vehicle_breaks": [
        {"vehicle_id": 3, "earliest": 10,  "latest": 20, "duration": 5, "allowed_waypoints": [9]},
        {"vehicle_id": 4, "earliest": 10, "latest": 20, "duration": 5, "allowed_waypoints": [9]},
    ],

}

Graph (used as input) and build cost matrix  -----------------------------

In [4]:
# 圖（你的 GRAPH）
GRAPH = {
    0:  {"edges":[1],         "weights":[1]},
    1:  {"edges":[4],         "weights":[2]},
    2:  {"edges":[0],         "weights":[1]},
    3:  {"edges":[6],         "weights":[2]},
    4:  {"edges":[3,7],       "weights":[2,2]},
    5:  {"edges":[6],         "weights":[2]},
    6:  {"edges":[7,11],      "weights":[2,2]},
    7:  {"edges":[8],         "weights":[2]},
    8:  {"edges":[2,9,14],    "weights":[4,2,2]},
    9:  {"edges":[37],        "weights":[10]},
    10: {"edges":[11],        "weights":[1]},
    11: {"edges":[10,12,17],  "weights":[1,1,2]},
    12: {"edges":[11],        "weights":[1]},
    13: {"edges":[14],        "weights":[1]},
    14: {"edges":[13,15,20],  "weights":[1,1,2]},
    15: {"edges":[14],        "weights":[1]},
    16: {"edges":[17],        "weights":[1]},
    17: {"edges":[16,18,23],  "weights":[1,1,2]},
    18: {"edges":[17],        "weights":[1]},
    19: {"edges":[20],        "weights":[1]},
    20: {"edges":[19,21,26],  "weights":[1,1,2]},
    21: {"edges":[20],        "weights":[1]},
    22: {"edges":[23],        "weights":[1]},
    23: {"edges":[22,24,29],  "weights":[1,1,2]},
    24: {"edges":[23],        "weights":[1]},
    25: {"edges":[26],        "weights":[1]},
    26: {"edges":[25,27,32],  "weights":[1,1,2]},
    27: {"edges":[26],        "weights":[1]},
    28: {"edges":[29],        "weights":[1]},
    29: {"edges":[28,30,35],  "weights":[1,1,2]},
    30: {"edges":[29],        "weights":[1]},
    31: {"edges":[32],        "weights":[1]},
    32: {"edges":[31,33,36],  "weights":[1,1,2]},
    33: {"edges":[32],        "weights":[1]},
    34: {"edges":[5],         "weights":[10]},
    35: {"edges":[34],        "weights":[2]},
    36: {"edges":[35],        "weights":[4]},
    37: {"edges":[36],        "weights":[2]},
}

In [ ]:
# 參數展開
factory_open  = CONFIG["factory_open"]
factory_close = CONFIG["factory_close"]
pickup_node   = CONFIG["pickup_node"]
stations      = CONFIG["stations"]

# 只用 Depot(0) + 取貨點 + 站點 這些 target 求解（決定成本矩陣的列/欄順序）
target_locations = np.array(sorted(set([0, pickup_node] + stations + CONFIG["charging_spots"])), dtype=np.int32)

# 建 WaypointMatrix 與成本/時間矩陣（時間=成本的初始做法）
wmat = build_waypoint_graph(GRAPH)
cost_matrix = wmat.compute_cost_matrix(target_locations)
transit_time_matrix = cost_matrix.copy(deep=True)

In [6]:
# cost_matrix[i][j] 表示從 target_locations[i] → target_locations[j] 的成本
index_to_node = {i: n for i, n in enumerate(target_locations)}
node_to_index = {n: i for i, n in enumerate(target_locations)}

In [7]:
# offsets, edges, weights_cost = graph_to_csr_arrays(GRAPH)   # 取 CSR 三元組
# edge_time_weights = weights_cost                            # 目前時間=成本

Add more station for delivery (by distance - closer first) -----------------------------

In [8]:
num_items = CONFIG["num_items"]

if CONFIG["use_nearby_split"]:
    deliveries, eta_map = choose_deliveries_near_first(
        stations=stations,
        num_items=num_items,
        pickup_node=pickup_node,
        target_locations=target_locations,
        cost_matrix=cost_matrix,     # 已有 cost_matrix，這裡就不需要再傳 waypoint_matrix
        waypoint_matrix=wmat,
    )
    if CONFIG["verbose_split"]:
        from collections import Counter
        print("每站件數：", Counter(deliveries))
        print("ETA(取貨→站)：", {s: round(eta_map[s], 2) for s in sorted(eta_map)})
else:
    # 不使用自動分配時，你可以自己指定 deliveries（下面只是示範）
    deliveries = stations[:num_items]

transport_df = cudf.DataFrame({
    "pickup_location":        [pickup_node] * num_items,                 # 原始 id
    "delivery_location":      deliveries,                                # 原始 id
    "order_demand":           [1] * num_items,
    "earliest_pickup":        [factory_open]  * num_items,
    "latest_pickup":          [factory_close] * num_items,
    "pickup_service_time":    [CONFIG["pickup_service_time"]] * num_items,
    "earliest_delivery":      [factory_open]  * num_items,
    "latest_delivery":        [factory_close] * num_items,
    "delivery_service_time":  [CONFIG["delivery_service_time"]] * num_items,
})

print("\n=== transport_df ===")
print_all(transport_df)


每站件數： Counter({10: 1, 12: 1, 13: 1, 15: 1, 16: 1, 18: 1, 19: 1, 21: 1, 22: 1, 24: 1, 25: 1, 27: 1, 28: 1, 30: 1, 31: 1, 33: 1})
ETA(取貨→站)： {10: 9.0, 12: 9.0, 13: 9.0, 15: 9.0, 16: 11.0, 18: 11.0, 19: 11.0, 21: 11.0, 22: 13.0, 24: 13.0, 25: 13.0, 27: 13.0, 28: 15.0, 30: 15.0, 31: 15.0, 33: 15.0}

=== transport_df ===
    pickup_location  delivery_location  order_demand  earliest_pickup  latest_pickup  pickup_service_time  earliest_delivery  latest_delivery  delivery_service_time
0                 1                 10             1                0            400                    0                  0              400                      0
1                 1                 12             1                0            400                    0                  0              400                      0
2                 1                 13             1                0            400                    0                  0              400                      0
3                 1   

In [9]:
# robot_df = cudf.DataFrame({
#     "robot_ids": CONFIG["robots"]["ids"],
#     "carrying_capacity": CONFIG["robots"]["capacity"],
# }).set_index("robot_ids")

# print("\n=== robot_df ===")
# print_all(robot_df)


robot_df = cudf.DataFrame({
    "robot_ids": CONFIG["robots"]["ids"],
    "carrying_capacity": CONFIG["robots"]["capacity"],
    "earliest_start": CONFIG["robots"].get("earliest_start", [CONFIG["factory_open"]] * len(CONFIG["robots"]["ids"])),
    "latest_end":     CONFIG["robots"].get("latest_end",     [CONFIG["factory_close"]] * len(CONFIG["robots"]["ids"])),
}).set_index("robot_ids")

print("\n=== robot_df ===")
print_all(robot_df)



=== robot_df ===
           carrying_capacity  earliest_start  latest_end
robot_ids                                               
0                          1               0          50
1                          1               1          50
2                          1               2          50
3                          1               3          50
4                          1               4          50
5                          1               5          50
6                          1               6         200
7                          1               7         150
8                          1               8         100


Setup data model and get solution  -----------------------------

In [10]:
dm = setup_data_model(
    target_locations=target_locations,
    transport_df=transport_df,
    robot_df=robot_df,
    cost_matrix=cost_matrix,
    transit_matrix=transit_time_matrix,  # 若你有更真實的時間矩陣，改成那一份
    factory_open=factory_open,
    factory_close=factory_close,
    must_return_to_depot=True,
    # min_vehicles=None,  # 如需強制至少出車數，填整數
)

In [11]:
# 假設 0 是 Depot，也可換成你的充電點 waypoint id（可多個）
vehicle_breaks = CONFIG["vehicle_breaks"]
apply_vehicle_breaks(dm, target_locations=target_locations, breaks=vehicle_breaks)

In [12]:
vehicle_breaks

[{'vehicle_id': 3,
  'earliest': 10,
  'latest': 20,
  'duration': 5,
  'allowed_waypoints': [9]},
 {'vehicle_id': 4,
  'earliest': 10,
  'latest': 20,
  'duration': 5,
  'allowed_waypoints': [9]}]

In [13]:
sol = solve_routing(dm, time_limit=CONFIG["solver_time_limit"])
print("\n=== Solve result ===")
print("Cost for the routing in time:", sol.get_total_objective())
print("Vehicle count to complete routing:", sol.get_vehicle_count())

route_raw = sol.get_route()
print("\n=== Raw route (from solver) ===")
print_all(route_raw)

# 展開前守門：location 應為 0..N-1（官方展開的必要條件）
assert_route_indexed(route_raw, n_locations=len(target_locations))


=== Solve result ===
Cost for the routing in time: 664.0
Vehicle count to complete routing: 9

=== Raw route (from solver) ===
    route  arrival_stamp  truck_id  location      type
0       0            0.0         0         0     Depot
1      11            1.0         0         1    Pickup
2      27           14.0         0        14  Delivery
3       0           46.0         0         0     Depot
4       0            1.0         1         0     Depot
5       3            2.0         1         1    Pickup
6      19           11.0         1         6  Delivery
7       0           47.0         1         0     Depot
8       0            2.0         2         0     Depot
9      14            3.0         2         1    Pickup
10     30           18.0         2        17  Delivery
11      0           48.0         2         0     Depot
12      0            3.0         3         0     Depot
13     13            4.0         3         1    Pickup
14      4            4.0         3         1   

In [14]:
route_with_idx = route_raw.copy(deep=True)
route_with_idx["order_array_index"] = route_with_idx["route"]

# index_map: cost-matrix index -> 原始 waypoint id
index_map = {k: v for k, v in enumerate(target_locations)}
route_with_idx["route"] = [
    index_map[idx] for idx in route_with_idx["location"].to_arrow().to_pylist()
]

print("\n=== Route with order_array_index (route = target/waypoint id) ===")
print_all(route_with_idx)


=== Route with order_array_index (route = target/waypoint id) ===
    route  arrival_stamp  truck_id  location      type  order_array_index
0       0            0.0         0         0     Depot                  0
1       1            1.0         0         1    Pickup                 11
2      27           14.0         0        14  Delivery                 27
3       0           46.0         0         0     Depot                  0
4       0            1.0         1         0     Depot                  0
5       1            2.0         1         1    Pickup                  3
6      15           11.0         1         6  Delivery                 19
7       0           47.0         1         0     Depot                  0
8       0            2.0         2         0     Depot                  0
9       1            3.0         2         1    Pickup                 14
10     31           18.0         2        17  Delivery                 30
11      0           48.0         2         0 

In [15]:
print("\n=== Target location level routes (per robot) ===")
uids = route_raw["truck_id"].unique().to_arrow().to_pylist()
for rid in uids:
    sub = route_raw[route_raw["truck_id"] == rid]["route"]
    print(f"\nRobot {rid}:")
    print_all(sub)


=== Target location level routes (per robot) ===

Robot 0:
0     0
1    11
2    27
3     0
Name: route, dtype: int32

Robot 1:
4     0
5     3
6    19
7     0
Name: route, dtype: int32

Robot 2:
8      0
9     14
10    30
11     0
Name: route, dtype: int32

Robot 3:
12     0
13    13
14     4
15    20
16    29
17     0
Name: route, dtype: int32

Robot 4:
18     0
19     9
20    25
21     0
Name: route, dtype: int32

Robot 5:
22     0
23    12
24    28
25     0
Name: route, dtype: int32

Robot 6:
26     0
27     2
28    18
29     7
30    23
31    10
32    26
33     8
34    24
35     0
Name: route, dtype: int32

Robot 7:
36     0
37     6
38    22
39     0
40    16
41     1
42    17
43     0
Name: route, dtype: int32

Robot 8:
44     0
45     5
46    21
47    15
48    31
49     0
Name: route, dtype: int32


In [16]:
print("\n=== Waypoint level routes (per robot) ===")
for rid in uids:
    sub = route_raw[route_raw["truck_id"] == rid]
    wp_route = wmat.compute_waypoint_sequence(target_locations, sub)  # 官方 API
    print(f"\nRobot {rid}:")
    print_all(wp_route)


=== Waypoint level routes (per robot) ===

Robot 0:
    waypoint_sequence waypoint_type
0                   0             w
1                   1        Pickup
2                   1             w
3                   4             w
4                   7             w
5                   8             w
6                  14             w
7                  20             w
8                  26             w
9                  27      Delivery
10                 27             w
11                 26             w
12                 32             w
13                 36             w
14                 35             w
15                 34             w
16                  5             w
17                  6             w
18                  7             w
19                  8             w
20                  2             w
21                  0         Depot

Robot 1:
    waypoint_sequence waypoint_type
0                   0             w
1                   1        Pickup
2

In [17]:
print("\n=== Waypoint level routes (per robot) ===")
for rid in uids:
    sub = route_raw[route_raw["truck_id"] == rid]
    wp_route = wmat.compute_waypoint_sequence(target_locations, sub)  # 官方 API
    print(f"\nRobot {rid}:")
    print_all(wp_route)


=== Waypoint level routes (per robot) ===

Robot 0:
    waypoint_sequence waypoint_type
0                   0             w
1                   1        Pickup
2                   1             w
3                   4             w
4                   7             w
5                   8             w
6                  14             w
7                  20             w
8                  26             w
9                  27      Delivery
10                 27             w
11                 26             w
12                 32             w
13                 36             w
14                 35             w
15                 34             w
16                  5             w
17                  6             w
18                  7             w
19                  8             w
20                  2             w
21                  0         Depot

Robot 1:
    waypoint_sequence waypoint_type
0                   0             w
1                   1        Pickup
2

---------------------

Get waypoint arrival time (to include w) / (cuopt only provide for the target location)-----------------------------

In [18]:
if CONFIG["compute_wp_times"]:
    offsets, edges, weights = graph_to_csr_arrays(GRAPH)
    wp_times = compute_waypoint_arrival_times(
        assignment=sol,
        waypoint_matrix=wmat,
        target_locations=target_locations,
        offsets=offsets, 
        edges=edges, 
        weights=weights,
        pickup_service_time=0,        # 服務時間：Pickup
        delivery_service_time=0,     # 服務時間：Delivery
        vehicle_breaks=vehicle_breaks,   
    )
    print("\n=== Waypoint arrival times (ALL) ===")
    print_all(wp_times)



=== Waypoint arrival times (ALL) ===
     truck_id  seq_index  waypoint waypoint_type  arrival_time
0           0          0         0         Depot           0.0
1           0          1         1        Pickup           1.0
2           0          2         4             w           3.0
3           0          3         7             w           5.0
4           0          4         8             w           7.0
5           0          5        14             w           9.0
6           0          6        20             w          11.0
7           0          7        26             w          13.0
8           0          8        27      Delivery          14.0
9           0          9        26             w          15.0
10          0         10        32             w          17.0
11          0         11        36             w          19.0
12          0         12        35             w          23.0
13          0         13        34             w          25.0
14          0    

In [19]:
import pandas as pd

df = wp_times.copy()

# 只保留必要欄位並確保型別
df["arrival_time"] = df["arrival_time"].astype(float)
df["waypoint"]     = df["waypoint"].astype(int)
df["truck_id"]     = df["truck_id"].astype(int)

# 以 (waypoint, arrival_time) 分組，抓出同時佔用（count > 1）
node_conflicts = (
    df.groupby(["waypoint", "arrival_time"])
      .agg(trucks=("truck_id", lambda s: tuple(sorted(set(s)))),
           n_trucks=("truck_id", "nunique"))
      .reset_index()
)

# 只保留有衝突的（同一時刻同一點被多台佔用）
node_conflicts = node_conflicts[node_conflicts["n_trucks"] > 1] \
                 .sort_values(["arrival_time", "waypoint"])

print(node_conflicts)


     waypoint  arrival_time     trucks  n_trucks
198        26          15.0     (0, 2)         2
167        17          16.0     (3, 5)         2
217        32          17.0     (0, 2)         2
188        23          18.0     (3, 5)         2
208        29          20.0     (3, 5)         2
210        29          22.0     (3, 5)         2
239        35          23.0     (0, 4)         2
240        35          24.0  (1, 3, 5)         3
227        34          25.0     (0, 4)         2
228        34          26.0  (1, 3, 5)         3
70          5          35.0     (0, 4)         2
71          5          36.0  (1, 3, 5)         3
86          6          37.0     (0, 4)         2
87          6          38.0  (1, 3, 5)         3
106         7          39.0     (0, 4)         2
107         7          40.0  (1, 3, 5)         3
126         8          41.0     (0, 4)         2
127         8          42.0  (1, 3, 5)         3
36          2          45.0     (0, 4)         2
9           0       

In [20]:
print(node_conflicts.to_string())

     waypoint  arrival_time     trucks  n_trucks
198        26          15.0     (0, 2)         2
167        17          16.0     (3, 5)         2
217        32          17.0     (0, 2)         2
188        23          18.0     (3, 5)         2
208        29          20.0     (3, 5)         2
210        29          22.0     (3, 5)         2
239        35          23.0     (0, 4)         2
240        35          24.0  (1, 3, 5)         3
227        34          25.0     (0, 4)         2
228        34          26.0  (1, 3, 5)         3
70          5          35.0     (0, 4)         2
71          5          36.0  (1, 3, 5)         3
86          6          37.0     (0, 4)         2
87          6          38.0  (1, 3, 5)         3
106         7          39.0     (0, 4)         2
107         7          40.0  (1, 3, 5)         3
126         8          41.0     (0, 4)         2
127         8          42.0  (1, 3, 5)         3
36          2          45.0     (0, 4)         2
9           0       

In [21]:
# 2) 做一個好讀的標籤：Pickup(1)、Delivery(31)、w(23)、Depot(0)…
df["pos_label"] = df["waypoint_type"].astype(str) + "(" + df["waypoint"].astype(str) + ")"

# 3) 同一台車在同一個時間若有多筆事件，合併成一格（避免 pivot 衝突）
events = (
    df.groupby(["arrival_time", "truck_id"], as_index=False)
      .agg(pos=("pos_label", lambda s: ",".join(dict.fromkeys(s))))
)

# 4) 轉成對照表：index=時間、columns=truck、values=位置
timeline = (
    events.pivot(index="arrival_time", columns="truck_id", values="pos")
          .sort_index()
          .fillna("-")
)

# 5) 完整列印（不出現 …）
print(timeline.to_string())

truck_id                 0             1             2             3             4             5             6             7             8
arrival_time                                                                                                                              
0.0               Depot(0)             -             -             -             -             -             -             -             -
1.0              Pickup(1)      Depot(0)             -             -             -             -             -             -             -
2.0                      -     Pickup(1)      Depot(0)             -             -             -             -             -             -
3.0                   w(4)             -     Pickup(1)      Depot(0)             -             -             -             -             -
4.0                      -          w(4)             -     Pickup(1)      Depot(0)             -             -             -             -
5.0                   w(7) 

-----------------------------------------

Different method to avoid conflict (but failed) -----------------------------

In [22]:
# import pandas as pd
# import numpy as np

# # 你希望最少錯開的時間（例如 1.0）
# gap = 1.0

# # node_conflicts 必須有欄位：trucks(tuple)、n_trucks
# # 把每筆衝突的 trucks=(t0,t1,...) 轉成相鄰對 [(t0,t1),(t1,t2),...]
# pairs = (node_conflicts["trucks"]
#          .apply(lambda t: list(zip(t, t[1:])))  # 相鄰車建立先後關係
#          .explode()
#          .dropna())

# # 組成邊表，並強制方向為小號→大號，避免環；再去重
# edges = (pd.DataFrame(pairs.tolist(), columns=["u","v"])
#            .astype(int))
# edges[["u","v"]] = np.sort(edges[["u","v"]].values, axis=1)
# edges = edges.drop_duplicates().reset_index(drop=True)

# # 節點集合
# nodes = sorted(set(edges["u"]).union(set(edges["v"])))

# # 每個節點的前驅清單（誰必須先於它出發）
# preds = edges.groupby("v")["u"].apply(list).to_dict()

# # 最長路徑 DP：offset[v] = max(offset[u] + gap)  for u in preds[v]
# offsets = {int(n): 0.0 for n in nodes}
# for v in nodes:  # 拓撲序可直接用數字遞增，因為我們已強制小號→大號
#     if v in preds:
#         offsets[v] = max(offsets[int(u)] + float(gap) for u in preds[v])

# # 排成表格看結果
# offset_df = (pd.DataFrame(sorted(offsets.items()), columns=["truck_id","suggested_delay"])
#                .astype({"truck_id":int, "suggested_delay":float}))
# print(offset_df.to_string(index=False))


In [23]:
# import numpy as np
# import pandas as pd
# import cudf

# # === A) 套用建議延後量到 earliest_start（承接上一段的 offset_df） ===
# # offset_df 需包含: ["truck_id", "suggested_delay"]
# robot_df2 = robot_df.copy()
# if "earliest_start" not in robot_df2.columns:
#     robot_df2["earliest_start"] = np.int32(0)

# for _, r in offset_df.iterrows():
#     tid   = int(r["truck_id"])
#     delay = float(r["suggested_delay"])
#     if tid in robot_df2.index:
#         base = int(robot_df2.loc[tid, "earliest_start"])
#         robot_df2.loc[tid, "earliest_start"] = np.int32(max(base, int(np.ceil(delay))))

# print("\n[updated earliest_start]")
# print(robot_df2.to_pandas().to_string())

# # === B) 用 solve.py 的函式重建 DM 並重解 ===
# cost_matrix = wmat.compute_cost_matrix(target_locations)
# dm2 = setup_data_model(
#     target_locations=target_locations,
#     transport_df=transport_df,
#     robot_df=robot_df2,                   # ← 已更新 earliest_start
#     cost_matrix=cost_matrix,
#     transit_matrix=None,                  # 若有時間矩陣可換成那份
#     factory_open=CONFIG["factory_open"],
#     factory_close=CONFIG["factory_close"],
#     must_return_to_depot=True,
# )
# sol2 = solve_routing(dm2, time_limit=CONFIG["solver_time_limit"])

# # === C) 重算 wp_times2 ===
# offsets_csr, edges_csr, weights_csr = graph_to_csr_arrays(GRAPH)
# wp_times2 = compute_waypoint_arrival_times(
#     assignment=sol2,
#     waypoint_matrix=wmat,
#     target_locations=target_locations,
#     offsets=offsets_csr, edges=edges_csr, weights=weights_csr,
#     pickup_service_time=0,                # 依你的設定
#     delivery_service_time=0,
# )

# # === D) 再用「節點衝突」檢查一次（你的原始作法）===
# df2 = wp_times2 if isinstance(wp_times2, pd.DataFrame) else wp_times2.to_pandas()
# df2["arrival_time"] = df2["arrival_time"].astype(float)
# df2["waypoint"]     = df2["waypoint"].astype(int)
# df2["truck_id"]     = df2["truck_id"].astype(int)

# node_conflicts2 = (
#     df2.groupby(["waypoint","arrival_time"])
#        .agg(trucks=("truck_id", lambda s: tuple(sorted(set(s)))),
#             n_trucks=("truck_id","nunique"))
#        .reset_index()
# )
# node_conflicts2 = node_conflicts2[node_conflicts2["n_trucks"] > 1] \
#                    .sort_values(["arrival_time","waypoint"])

# print("\n[conflicts after re-solve]")
# print("None" if len(node_conflicts2)==0 else node_conflicts2.to_string(index=False))


In [24]:
# import pandas as pd
# import numpy as np

# gap = 1.0  # 你要的最小錯開時間

# def compute_incremental_bump_from_conflicts(node_conflicts: pd.DataFrame, gap: float = 1.0) -> pd.DataFrame:
#     """
#     針對目前的節點衝突，把 trucks=(t0,t1,...) 展開成相鄰對 (t0,t1),(t1,t2)...
#     固定用小號→大號當作 leader→follower。
#     回傳每個 follower 需要「再增加」的 earliest_start（extra_delay），= 出現次數 × gap。
#     """
#     if node_conflicts is None or len(node_conflicts) == 0:
#         return pd.DataFrame(columns=["truck_id", "extra_delay"])

#     pairs = (node_conflicts["trucks"]
#              .apply(lambda t: list(zip(t, t[1:])))   # 相鄰車
#              .explode().dropna())
#     if pairs.empty:
#         return pd.DataFrame(columns=["truck_id", "extra_delay"])

#     edges = pd.DataFrame(pairs.tolist(), columns=["u","v"]).astype(int)
#     # 小號→大號，避免環也固定「誰是 follower」
#     edges[["u","v"]] = np.sort(edges[["u","v"]].values, axis=1)

#     # 計次（每出現一次，就需要多錯開 gap）
#     cnt = edges.groupby("v").size().rename("count").reset_index()
#     cnt["extra_delay"] = cnt["count"].astype(float) * float(gap)
#     out = cnt[["v","extra_delay"]].rename(columns={"v":"truck_id"}).sort_values("truck_id")
#     return out

# # ← 這裡用你剛印出的 node_conflicts2
# extra_df = compute_incremental_bump_from_conflicts(node_conflicts2, gap=gap)
# print("\n[incremental bump (extra earliest_start needed)]")
# print("None" if len(extra_df)==0 else extra_df.to_string(index=False))


In [25]:
# import numpy as np
# import cudf

# # 1) 把剛算出的 extra 套到 earliest_start（加總，而不是取 max）
# robot_df3 = robot_df2.copy()
# for _, r in extra_df.iterrows():
#     tid  = int(r["truck_id"])
#     bump = int(np.ceil(float(r["extra_delay"])))
#     if tid in robot_df3.index:
#         robot_df3.loc[tid, "earliest_start"] = np.int32(int(robot_df3.loc[tid, "earliest_start"]) + bump)

# print("\n[updated earliest_start (after bump)]")
# print(robot_df3.to_pandas().to_string())

# # 2) 用 solve.py 的函式重建 DM 並重解
# cost_matrix = wmat.compute_cost_matrix(target_locations)
# dm3 = setup_data_model(
#     target_locations=target_locations,
#     transport_df=transport_df,
#     robot_df=robot_df3,                  # ← 已加上額外延後
#     cost_matrix=cost_matrix,
#     transit_matrix=None,                 # 若有時間矩陣就換那份
#     factory_open=CONFIG["factory_open"],
#     factory_close=CONFIG["factory_close"],
#     must_return_to_depot=True,
# )
# sol3 = solve_routing(dm3, time_limit=CONFIG["solver_time_limit"])

# # 3) 重算 wp_times3
# offs, edg, wgt = graph_to_csr_arrays(GRAPH)
# wp_times3 = compute_waypoint_arrival_times(
#     assignment=sol3,
#     waypoint_matrix=wmat,
#     target_locations=target_locations,
#     offsets=offs, edges=edg, weights=wgt,
#     pickup_service_time=0,
#     delivery_service_time=0,
# )

# # 4) 再檢查節點衝突
# df3 = wp_times3 if isinstance(wp_times3, pd.DataFrame) else wp_times3.to_pandas()
# df3["arrival_time"] = df3["arrival_time"].astype(float)
# df3["waypoint"]     = df3["waypoint"].astype(int)
# df3["truck_id"]     = df3["truck_id"].astype(int)

# node_conflicts3 = (
#     df3.groupby(["waypoint","arrival_time"])
#        .agg(trucks=("truck_id", lambda s: tuple(sorted(set(s)))),
#             n_trucks=("truck_id","nunique"))
#        .reset_index()
# )
# node_conflicts3 = node_conflicts3[node_conflicts3["n_trucks"] > 1] \
#                    .sort_values(["arrival_time","waypoint"])

# print("\n[conflicts after bump & re-solve]")
# print("None" if len(node_conflicts3)==0 else node_conflicts3.to_string(index=False))


In [ ]:
# import numpy as np
# import pandas as pd
# import cudf

# # 1) 衝突 → 每台車的最小延後量（以 gap 為單位）
# def compute_start_offsets_from_conflicts(node_conflicts: pd.DataFrame,
#                                          gap: float = 1.0,
#                                          all_robot_ids=None) -> dict:
#     """
#     node_conflicts: 必須有欄位 trucks (tuple)、n_trucks
#     回傳: {truck_id: offset(浮點)}，代表 earliest_start 需至少延後 offset
#     """
#     if node_conflicts is None or len(node_conflicts) == 0:
#         # 沒衝突 → 全部延後 0
#         return {int(t): 0.0 for t in (all_robot_ids or [])}

#     # 每個衝突，把 (t0,t1,t2,...) 變成 [(t0,t1), (t1,t2), ...]
#     pairs = (node_conflicts["trucks"]
#              .apply(lambda t: list(zip(t, t[1:])))  # 相鄰車建立先後關係
#              .explode().dropna())
#     if pairs is None or len(pairs) == 0:
#         return {int(t): 0.0 for t in (all_robot_ids or [])}

#     edf = pd.DataFrame(pairs.tolist(), columns=["u","v"]).drop_duplicates()
#     edf["u"] = edf["u"].astype(int); edf["v"] = edf["v"].astype(int)

#     # 節點集合
#     nodes = set(edf["u"]).union(set(edf["v"]))
#     if all_robot_ids is not None:
#         nodes.update(int(x) for x in all_robot_ids)
#     nodes = sorted(nodes)

#     # 只建「小號→大號」的邊（若輸入剛好反了也修正回來），避免環
#     edf = pd.DataFrame(
#         [(min(u,v), max(u,v)) for u,v in edf[["u","v"]].to_numpy()],
#         columns=["u","v"]
#     ).drop_duplicates()

#     # 最長路徑 DP（拓樸序可直接用車號遞增）
#     offsets = {int(n): 0.0 for n in nodes}
#     for v in nodes:
#         preds = edf.loc[edf["v"] == v, "u"].to_numpy()
#         if preds.size:
#             offsets[v] = max(offsets[int(u)] + float(gap) for u in preds)

#     return offsets

# # 2) 把延後量套用到 robot_df['earliest_start']（取天花板、維持 int32）
# def apply_earliest_start_offsets(robot_df: cudf.DataFrame, offsets: dict) -> cudf.DataFrame:
#     rdf = robot_df.copy()
#     if "earliest_start" not in rdf.columns:
#         rdf["earliest_start"] = np.int32(0)
#     for tid, delay in offsets.items():
#         if tid in rdf.index:
#             base = int(rdf.loc[tid, "earliest_start"])
#             # 取 ceil，確保至少延後 gap 單位；也可加個小 epsilon 再取整
#             rdf.loc[tid, "earliest_start"] = np.int32(int(np.ceil(max(base, delay))))
#     return rdf

# # 3) （可選）一鍵套用 + 重解 + 再檢查
# def stagger_once_and_resolve(
#     *, node_conflicts: pd.DataFrame, robot_df: cudf.DataFrame,
#     gap: float,
#     GRAPH, waypoint_matrix,
#     transport_df, target_locations,
#     factory_open, factory_close, time_limit,
#     pickup_service_time=0, delivery_service_time=0,
# ):
#     # 算每台車 offset
#     all_ids = robot_df.index.to_pandas().tolist()
#     offsets = compute_start_offsets_from_conflicts(node_conflicts, gap=gap, all_robot_ids=all_ids)
#     # 印出建議延後量
#     off_df = pd.DataFrame(sorted(offsets.items()), columns=["truck_id","suggested_delay"])
#     print("\n[suggested earliest_start offsets]")
#     print(off_df.to_string(index=False))

#     # 套用到 robot_df
#     robot_df2 = apply_earliest_start_offsets(robot_df, offsets)
#     print("\n[updated earliest_start]")
#     print(robot_df2.to_pandas().to_string())

#     # 重解一次（維持你現有的流程）
#     dm = setup_data_model(
#         target_locations=target_locations,
#         transport_df=transport_df,
#         robot_df=robot_df2,
#         cost_matrix=waypoint_matrix.compute_cost_matrix(target_locations),
#         transit_matrix=None,
#         factory_open=factory_open,
#         factory_close=factory_close,
#         must_return_to_depot=True,
#     )
#     sol2 = solve_routing(dm, time_limit=time_limit)

#     # 重算 wp_times
#     offsets_csr, edges_csr, weights_csr = graph_to_csr_arrays(GRAPH)
#     wp_times2 = compute_waypoint_arrival_times(
#         assignment=sol2,
#         waypoint_matrix=waypoint_matrix,
#         target_locations=target_locations,
#         offsets=offsets_csr, edges=edges_csr, weights=weights_csr,
#         pickup_service_time=pickup_service_time,
#         delivery_service_time=delivery_service_time,
#     )

#     # 再檢一次衝突（用你原本的作法）
#     df2 = wp_times2 if isinstance(wp_times2, pd.DataFrame) else wp_times2.to_pandas()
#     df2["arrival_time"] = df2["arrival_time"].astype(float)
#     df2["waypoint"]     = df2["waypoint"].astype(int)
#     df2["truck_id"]     = df2["truck_id"].astype(int)

#     nc2 = (df2.groupby(["waypoint","arrival_time"])
#               .agg(trucks=("truck_id", lambda s: tuple(sorted(set(s)))),
#                    n_trucks=("truck_id","nunique"))
#               .reset_index())
#     nc2 = nc2[nc2["n_trucks"] > 1].sort_values(["arrival_time","waypoint"])
#     print("\n[conflicts after one stagger]")
#     if len(nc2) == 0:
#         print("None")
#     else:
#         print(nc2.to_string(index=False))

#     return sol2, wp_times2, robot_df2, off_df, nc2

In [ ]:
# max_rounds = 5
# round_i = 0
# while len(conf2) > 0 and round_i < max_rounds:
#     print(f"\n[stagger loop] round {round_i+1}, conflicts={len(conf2)}")

#     sol2, wp_times2, robot_df2, off_df, conf2 = stagger_once_and_resolve(
#         node_conflicts=conf2,
#         robot_df=robot_df2,           # 用上一輪已更新過 earliest_start 的 robot_df2
#         gap=1.0,                      # 需要更保守就用 2.0 或更大
#         GRAPH=GRAPH,
#         waypoint_matrix=wmat,
#         transport_df=transport_df,
#         target_locations=target_locations,
#         factory_open=CONFIG["factory_open"],
#         factory_close=CONFIG["factory_close"],
#         time_limit=CONFIG["solver_time_limit"],
#         pickup_service_time=0,
#         delivery_service_time=0,
#     )
#     round_i += 1

# print("\n[final conflicts]")
# print("None" if len(conf2)==0 else conf2.to_string(index=False))
# print("\n[final earliest_start]")
# print(robot_df2.to_pandas().to_string())

----------------------------------------

In [ ]:
# # NEW: 將 (waypoint, arrival_time) 的衝突清單轉為「等待虛擬訂單」
# def make_wait_orders_from_conflicts(wp_times_pd, node_conflicts_pd, *,
#                                     gap=1.0, tol=0.0, time_scale=1):
#     """
#     wp_times_pd: pandas DataFrame（包含 truck_id, waypoint, arrival_time, seq_index, waypoint_type）
#     node_conflicts_pd: pandas DataFrame（包含 waypoint, arrival_time, trucks(tuple), n_trucks）
#     回傳 pandas DataFrame：包含 pickup/delivery 同點 + pickup_service_time=等待秒數 的虛擬單
#     """
#     import numpy as np
#     import pandas as pd

#     # 依車整理時間序列
#     gdict = {int(tid): g.sort_values("arrival_time").reset_index(drop=True)
#              for tid, g in wp_times_pd.groupby("truck_id", sort=True)}

#     waits = []
#     for _, row in node_conflicts_pd.iterrows():
#         wp = int(row["waypoint"])
#         t  = float(row["arrival_time"])
#         trucks = sorted(set(row["trucks"]))  # e.g. (0,2) -> [0,2]
#         if len(trucks) <= 1:
#             continue
#         leader, followers = trucks[0], trucks[1:]  # 讓最小 id 先走

#         for j, tid in enumerate(followers, start=1):
#             g = gdict[int(tid)]

#             # 找出該車在此 (wp,t) 的 index（若 tol>0，用容忍）
#             if tol > 0:
#                 mask = (g["waypoint"].astype(int).eq(wp) &
#                         (np.abs(g["arrival_time"] - t) <= tol/2))
#             else:
#                 mask = (g["waypoint"].astype(int).eq(wp) & g["arrival_time"].eq(t))
#             idx = np.flatnonzero(mask.values)
#             if len(idx) == 0:
#                 # 萬一浮點誤差找不到，就取最接近的
#                 idx = [int(np.argmin(np.abs(g["arrival_time"].values - t)))]
#             i = int(idx[0])

#             # 上一個節點（若 i==0 就在當前點等）
#             if i == 0:
#                 prev_wp = int(g.loc[i, "waypoint"])
#                 t_prev  = float(g.loc[i, "arrival_time"])
#             else:
#                 prev_wp = int(g.loc[i-1, "waypoint"])
#                 t_prev  = float(g.loc[i-1, "arrival_time"])

#             # 等待時間：推到 t，再+ j*gap 讓同群跟車錯開
#             base_wait = max(0.0, t - t_prev)
#             wait = base_wait + j * gap

#             waits.append({
#                 "pickup_location":       prev_wp,
#                 "delivery_location":     prev_wp,
#                 "order_demand":          0,
#                 "earliest_pickup":       int(round((t_prev) * time_scale)),
#                 "latest_pickup":         int(round((t_prev) * time_scale)),
#                 "pickup_service_time":   int(round((wait)   * time_scale)),  # 在上一點等待
#                 "earliest_delivery":     int(round((t_prev + wait) * time_scale)),
#                 "latest_delivery":       int(round((t_prev + wait) * time_scale)),
#                 "delivery_service_time": 0,
#             })

#     return pd.DataFrame(waits)


In [ ]:
# # NEW: 依等待虛擬單重建 DM 並重解一次
# def re_solve_with_wait_orders(
#     *,
#     GRAPH,
#     waypoint_matrix,              # distance_engine.WaypointMatrix（建議與 GRAPH 同源）
#     transport_df, robot_df,
#     target_locations,
#     virtual_orders_pd,            # pandas DF（make_wait_orders_from_conflicts 的輸出）
#     factory_open, factory_close,
#     time_limit,
# ):
#     import numpy as np
#     import cudf

#     # 1) 若無虛擬單，直接回傳原資料
#     if virtual_orders_pd is None or len(virtual_orders_pd) == 0:
#         return None, None, target_locations, transport_df

#     # 2) append 虛擬單
#     transport_df2 = cudf.concat([transport_df, cudf.from_pandas(virtual_orders_pd)], ignore_index=True)

#     # 3) 確保 target_locations 含所有 prev_wp
#     prev_wps = virtual_orders_pd["pickup_location"].unique()
#     target_locations2 = np.unique(np.concatenate([target_locations, prev_wps])).astype(int)

#     # 4) 以新的 target_locations2 重算矩陣
#     cost_matrix2   = waypoint_matrix.compute_cost_matrix(target_locations2)
#     # 如果你有實際 transit time 矩陣，這裡一併重算；否則傳 None
#     transit_matrix2 = None

#     # 5) 重建 DM & 重解
#     dm2 = setup_data_model(
#         target_locations=target_locations2,
#         transport_df=transport_df2,
#         robot_df=robot_df,
#         cost_matrix=cost_matrix2,
#         transit_matrix=transit_matrix2,
#         factory_open=factory_open,
#         factory_close=factory_close,
#         must_return_to_depot=True,
#     )
#     sol2 = solve_routing(dm2, time_limit=time_limit)

#     # 6) 用同一版 GRAPH/CSR 計算 wp_times
#     offsets, edges, weights = graph_to_csr_arrays(GRAPH)
#     wp_times2 = compute_waypoint_arrival_times(
#         assignment=sol2,
#         waypoint_matrix=waypoint_matrix,
#         target_locations=target_locations2,
#         offsets=offsets, edges=edges, weights=weights,
#         pickup_service_time=0,
#         delivery_service_time=0,
#     )
#     return sol2, wp_times2, target_locations2, transport_df2


In [ ]:
# # A) 產生等待虛擬單（接你現有的 node_conflicts）
# virtual_orders_pd = make_wait_orders_from_conflicts(
#     wp_times_pd = wp_times if isinstance(wp_times, pd.DataFrame) else wp_times.to_pandas(),
#     node_conflicts_pd = node_conflicts,
#     gap=1.0,    # 同一時刻同一節點，跟在後面的車每台多等 1 單位
#     tol=0.0,    # 如要容忍浮點誤差，可設 1e-6 ~ 1e-3
#     time_scale=1
# )

# # B) 重排一次（會自動 append 虛擬單、補 target_locations、重解、重算 wp_times）
# sol2, wp_times2, target_locations2, transport_df2 = re_solve_with_wait_orders(
#     GRAPH=GRAPH,
#     waypoint_matrix=wmat,             # 確保與 GRAPH 同源
#     transport_df=transport_df,
#     robot_df=robot_df,
#     target_locations=target_locations,
#     virtual_orders_pd=virtual_orders_pd,
#     factory_open=CONFIG["factory_open"],
#     factory_close=CONFIG["factory_close"],
#     time_limit=CONFIG["solver_time_limit"],
# )

# # # C) 再跑一次你的衝突檢查，確認是否清除
# df2 = wp_times2 if isinstance(wp_times2, pd.DataFrame) else wp_times2.to_pandas()
# node_conflicts2 = (
#     df2.groupby(["waypoint","arrival_time"])
#        .agg(trucks=("truck_id", lambda s: tuple(sorted(set(s)))),
#             n_trucks=("truck_id","nunique"))
#        .reset_index()
# )
# node_conflicts2 = node_conflicts2[node_conflicts2["n_trucks"] > 1] \
#                    .sort_values(["arrival_time","waypoint"])
# print(node_conflicts2.to_string(index=False))


-------------------------------------------------------

Visualizatoin -----------------------------

In [28]:
MY_COORDS_DICT = {
    0: (5, 14),
    1: (4, 14),
    2: (6, 14),
    3: (2, 12),
    4: (4, 12),
    5: (0, 10),
    6: (2, 10),
    7: (4, 10),
    8: (6, 10),
    9: (8, 10),
    10:(1, 8),
    11:(2, 8),
    12:(3, 8),
    13:(5, 8),
    14:(6, 8),  
    15:(7, 8),
    16:(1, 6),
    17:(2, 6),
    18:(3, 6),
    19:(5, 6),
    20:(6, 6),
    21:(7, 6),
    22:(1,4),
    23:(2, 4),
    24:(3, 4),
    25:(5, 4),
    26:(6, 4),
    27:(7, 4),
    28:(1,2),
    29:(2, 2),
    30:(3, 2),
    31:(5,2),
    32:(6, 2),
    33:(7, 2),
    34:(0, 0),
    35:(2, 0),
    36:(6, 0),
    37:(8, 0)          
}

In [29]:
schedule_df = wp_times
coords = MY_COORDS_DICT  # {0:(x0,y0), 1:(x1,y1), ...}

fig = build_animation_figure(
    schedule_df=schedule_df,
    graph=GRAPH,
    coords=coords,
    service_time_map={
        "Pickup":   CONFIG["pickup_service_time"],   # 你模型的服務時間
        "Delivery": CONFIG["delivery_service_time"],
        "Depot":    0.0,
        "w":        0.0,
        "Break":    5.0,   # ★ 把 add_vehicle_break 設的 duration 放這裡
    },
    dt=0.5,
    frame_ms=180,
    slider_stride=2,
    title="AGV 動態路徑動畫",
    edge_color="lightblue",
    node_color="lightblue",
    node_size=18,
)
fig.show()


In [ ]:
# # schedule_df = wp_times_noconf if 'wp_times_noconf' in locals() else wp_times
# schedule_df = wp_times2
# coords = MY_COORDS_DICT  # {0:(x0,y0), 1:(x1,y1), ...}

# fig = build_animation_figure(
#     schedule_df=schedule_df,
#     graph=GRAPH,
#     coords=coords,
#     service_time_map={"Pickup":0.0, "Delivery":0.0, "Depot":0.0, "w":0.0},
#     dt=0.5,
#     frame_ms=180,
#     slider_stride=2,
#     title="AGV 動態路徑動畫",
#     edge_color="lightblue",
#     node_color="lightblue",
#     node_size=18,
#     # agv_color_map 不傳 → 自動配色並顯示圖例
# )
# fig.show()

-------------------------------------------------